In [2]:
import pandas as pd
import numpy as np
import os 
import geopandas as gpd
import plotly.express as px

In [29]:
mf = gpd.read_file('./Marine_forest/Marine_forest.shp')
count = len(mf[(mf['면적'] == mf['구역']) & (mf['면적'] == mf['규모'])])
print(count)
mf.drop(columns=['구역','규모'],inplace=True)
col_names = {'시도명': 'province','시군구명':'city','시군구코드':'city_code','위도':'latitude','경도':'longitude','마을어장':'town','면적':'area'}
mf.rename(columns=col_names, inplace=True)
#기본 칼럼인 위도 경도는 오류가 있었음
mf['latitude'] = mf['geometry'].y
mf['longitude'] = mf['geometry'].x

219


In [48]:
title = '2013~2022년 바다숲 조성사업 현황'
buff = mf
buff = buff.to_crs(epsg=32652)
center = {'lat': 35.8, 'lon': buff['longitude'].median()}
fig = px.scatter_mapbox(buff, lat=buff['latitude'], lon=buff['longitude'], size=buff['area'],
                        color=buff['area'], size_max=50, zoom=5, center=center, title =title)
fig.update_layout(mapbox_style="open-street-map", margin={"r":10,"t":70,"l":20,"b":20},
                  width=1200 , height=800)
fig.show()

In [97]:
mf_2013 = mf.loc[mf['시설년도']==2013]
mf_2014 = mf.loc[mf['시설년도']==2014]
mf_2015 = mf.loc[mf['시설년도']==2015]
mf_2016 = mf.loc[mf['시설년도']==2016]
mf_2017 = mf.loc[mf['시설년도']==2017]
mf_2018 = mf.loc[mf['시설년도']==2018]
mf_2019 = mf.loc[mf['시설년도']==2019]
mf_2020 = mf.loc[mf['시설년도']==2020]

title = '2018년 해양지형정보 2013~2018바다숲'
buff = mf_2013
buff = buff.to_crs(epsg=32652)

center = {'lat': 35.8, 'lon': buff['longitude'].median()}
fig = px.scatter_mapbox(buff, lat=buff['latitude'], lon=buff['longitude'], size=buff['area'],
                        color=buff['area'], size_max=50, zoom=5, center=center, title =title)
fig.update_layout(mapbox_style="open-street-map", margin={"r":10,"t":70,"l":20,"b":20},
                  width=1200 , height=800)
fig.show()



## 2010년~ 2020년 해양환경 변화 시각화

In [98]:
# Ocean_data 폴더 내의 모든 CSV 파일 목록을 가져옵니다.
folder_path = 'Ocean_data'
all_files = [f for f in os.listdir(folder_path) if f.endswith('.csv') and f.startswith('19') or f.startswith('20')]

# 각 파일을 순차적으로 읽어 데이터 프레임 리스트에 추가합니다.
dfs = []
for filename in all_files:
    file_path = os.path.join(folder_path, filename)
    df = pd.read_csv(file_path)
    dfs.append(df)

# 모든 데이터 프레임을 하나로 합칩니다.
merged_data = pd.concat(dfs, ignore_index=True)
##Longitude:위도 , Latitude: 경도
colnames = {'yyyy-mm-dd hh:mm:ss': '일시', 'Longitude [degrees_east]': 'Longitude', 'Latitude [degrees_north]': 'Latitude'}
merged_data.rename(columns=colnames, inplace=True)

merged_data = merged_data.copy()
#데이터 품질값이 1~2인 값만 남기고 나머지는 NaN으로 대체합니다.
qc_cols = [(idx, col) for idx, col in enumerate(merged_data.columns) if col.endswith('_qc')]
for idx, col in qc_cols:
    obs_df = merged_data.iloc[:, [idx - 1, idx]].copy()
    obs_df.iloc[~obs_df.iloc[:, 1].isin([1, 2]), 0] = np.nan
    col_value = merged_data.columns[idx - 1]
    merged_data[col_value] = obs_df.iloc[:, 0]
#qc칼럼 제거
qc_cols = [col for col in merged_data.columns if col.endswith('_qc')]

merged_data['일시'] = pd.to_datetime(merged_data['일시'],format='mixed')
merged_data['year'] = merged_data['일시'].dt.year
merged_data['month'] = merged_data['일시'].dt.month
merged_data['quarter'] = merged_data['일시'].dt.quarter
merged_data['year_quarter'] = merged_data['year'].astype(str) + merged_data['quarter'].astype(str)

merged_data.drop(columns=qc_cols+['일시','project_name', 'bot.depth[m]', 'dep_flag', '유효시간','전체수심(m)'], inplace=True)

In [99]:
missing_values_per_column = merged_data.isnull().sum()
print(missing_values_per_column)


station               0
Longitude             0
Latitude              0
수심(m)                 1
수심(text)           9536
총질소[μg/L]          3231
염분[psu]             492
수온[℃]                15
클로로필-a[μg/L]         75
부유물질 농도[μg/L]     12768
용존산소[mg/L]           35
수소이온농도[무단위]          32
규산염[μg/L]          7365
아질산성질소[μg/L]          1
인산염인[μg/L]         7619
질산성질소[μg/L]        2940
투명도[m]            15969
용존무기질소[μg/L]          1
화학적산소요구량[mg/L]        1
암모니아성 질소[μg/L]        1
유분[mg/L]          32101
총인[μg/L]              1
부유물질 농도[mg/L]     19342
year                  0
month                 0
quarter               0
year_quarter          0
dtype: int64


In [100]:
surface_df = merged_data.loc[merged_data['수심(text)'] == '표층']
surface_df.drop(columns=['수심(m)','수심(text)'],inplace=True)
surface_df.isnull().sum()

C:\Users\우승우\AppData\Local\Temp\ipykernel_24900\3570654716.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



station               0
Longitude             0
Latitude              0
총질소[μg/L]          1615
염분[psu]             361
수온[℃]                 0
클로로필-a[μg/L]         47
부유물질 농도[μg/L]      1615
용존산소[mg/L]           31
수소이온농도[무단위]           4
규산염[μg/L]            31
아질산성질소[μg/L]          0
인산염인[μg/L]            9
질산성질소[μg/L]         205
투명도[m]               40
용존무기질소[μg/L]          0
화학적산소요구량[mg/L]        0
암모니아성 질소[μg/L]        0
유분[mg/L]          11404
총인[μg/L]              0
부유물질 농도[mg/L]      9796
year                  0
month                 0
quarter               0
year_quarter          0
dtype: int64

In [101]:
columns_to_plot = ['총질소[μg/L]', '총인[μg/L]', '용존무기질소[μg/L]','화학적산소요구량[mg/L]','암모니아성 질소[μg/L]']

fig = px.scatter(surface_df, x='year_quarter', y = columns_to_plot, title='Values over Year Quarter')
fig.show()

In [ ]:
low_df = merged_data.loc[merged_data['수심(text)'] == '저층']

In [92]:
columns_to_plot = ['총질소[μg/L]', '총인[μg/L]', '용존무기질소[μg/L]','화학적산소요구량[mg/L]','암모니아성 질소[μg/L]']

fig = px.scatter(low_df, x='year_quarter', y='총질소[μg/L]', title='Values over Year Quarter')
fig.show()

In [12]:
if not os.path.exists(folder_path):
    os.makedirs('./data/')
